In [24]:
import sys  
import os
sys.path.append(os.path.abspath(r"C:\Users\Jil Henry\Documents\Programming\Final Year Project\acne_diagnosis\python"))
sys.path.append(os.path.abspath(r"C:\Users\Jil Henry\Documents\Programming\Final Year Project\acne_diagnosis\python\api"))

In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from data_preprocessing import resize_rescale
import constants as Constants
import data_formatter as df

In [26]:
MODEL = tf.keras.models.load_model('./saved_models/8')

In [34]:
PARENT_FILE_PATH = './dataset/test/'
files = os.listdir(PARENT_FILE_PATH)

images_array = []
predictions = []
for file in files:
    pil_image = tf.keras.utils.load_img(PARENT_FILE_PATH + file,target_size=(224,224))
    image_array = tf.keras.preprocessing.image.img_to_array(pil_image)
    image_array = tf.expand_dims(image_array,0).numpy()
#     image_array = resize_rescale(image_array)
#     print(f"Image_Array: {image_array.shape}")
#     prediction = MODEL.predict(image_array).tolist()
#     predictions.append(prediction[0])
    images_array.append(image_array[0])
images_array = np.array(images_array)
type(images_array)

numpy.ndarray

In [35]:
predictions = MODEL.predict(images_array)
num_predictions = len(predictions)
print(f"Predictions: {predictions}")

Predictions: [[0.35111353 0.64888644]
 [0.22441503 0.775585  ]
 [0.27334276 0.7266572 ]
 [0.6193698  0.38063025]
 [0.9890593  0.01094069]]


In [36]:
print(f"Num of Predictions: {num_predictions}")
if num_predictions < 1:
    print(df.format_result(
        status='No Prediction',
        result_type=Constants.RESULT_TYPE_CONC))

pred_classes_code = np.array([], dtype='int64')
confidence_list_total = np.zeros_like(predictions[0])

if num_predictions == 1:
    #return{"prediction":str(predictions)}
    pred_class_code, confidence_list = df.format_prediction_result(
            predictions[0])
    #print(np.round(confidence_list.tolist()[pred_class_code],2))
    print( df.format_result(
            status='OK',
            result_type=Constants.RESULT_TYPE_CONC,
            pred_class=Constants.CLASS_NAMES[pred_class_code],
            confidence= np.round(confidence_list.tolist()[pred_class_code],2) * 100,
        ))
elif num_predictions > 1:
    for prediction in predictions:
        pred_class_code, confidence_list = df.format_prediction_result(
                prediction)
        pred_classes_code = np.append(pred_classes_code, pred_class_code)
        confidence_list_total += confidence_list

    print(f"Confidence Total = {confidence_list_total}")
    # Getting Number of  Most Occurring Class
    pred_classes_code_mode = np.bincount(pred_classes_code)
    max_num = np.max(pred_classes_code_mode)
    pred_classes_code_mode_num = [x for x in pred_classes_code_mode if x == max_num]
    # Return Results
    if len(pred_classes_code_mode_num) > 1:
        print(confidence_list_total)
        print( df.format_result(
              status='Inconclusive',
              result_type=Constants.RESULT_TYPE_INCONC
           ))
    else:
        index = pred_classes_code_mode.argmax()
        print(f"Predicted Index: {index}")
        confidence = np.round(((confidence_list_total[index] / num_predictions) * 100),2)
        pred_class = Constants.CLASS_NAMES[index]
        print( df.format_result(
                    status='OK',
                    result_type=Constants.RESULT_TYPE_CONC,
                    pred_class=pred_class,
                    confidence=confidence
                ))

Num of Predictions: 5
Confidence Total = [2.45 2.55]
Predicted Index: 1
{'status': 'OK', 'resultType': 'Conclusive', 'predClass': 'not-acne', 'confidence': 51.0}
